- MNIST数据集是经典图像数据集，包括10个类别(0到9)。每一张图片拉成向量表示，如下图784维向量作为第一层输入特征。

# Softmax分类

- softmax函数的本质就是将一个K 维的任意实数向量压缩（映射）成另一个K维的实数向量，其中向量中的每个元素取值都介于（0，1）之间，并且压缩后的K个值相加等于1(变成了概率分布)。
- 在选用Softmax做多分类时，可以根据值的大小来进行多分类的任务，如取权重最大的一维。
- 下面使用Pytorch定义一个多层网络(4个隐藏层，最后一层softmax概率归一化)，输出层为10正好对应10类。

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        # Flatten the data (n, 1, 28, 28) --> (n, 784)
        x = x.view(-1, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return F.log_softmax(self.l5(x), dim=1)
        #return self.l5(x)

In [4]:
def train(epoch):
    # 每次输入barch_idx个数据
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        # loss
        loss = F.nll_loss(output, target)
        loss.backward()
        # update
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    test_loss = 0
    correct = 0
    # 测试集
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target).data[0]
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for epoch in range(1,6):
    train(epoch)
    test()

Processing...
Done!


D:\software\conda\lib\site-packages\ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.296800
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.287799
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.283863
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.269646
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.156842


D:\software\conda\lib\site-packages\ipykernel_launcher.py:23: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
D:\software\conda\lib\site-packages\ipykernel_launcher.py:26: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.0243, Accuracy: 5630/10000 (56%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.555547
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.910738
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.488389
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.942790
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.677764

Test set: Average loss: 0.0073, Accuracy: 8570/10000 (85%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.597010
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.463823
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.239384
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.285246
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.338234

Test set: Average loss: 0.0053, Accuracy: 8999/10000 (89%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.293521
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.337644
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.242295
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.398434
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.182391

Test set: Average loss: 0.0035, Accuracy: 9357/10000 (93%)

Train Epoch: 5 [0/600